Data: 
Option prices and implied volatility from post-no-preference option chain dataset. data spans from 2019-current. collected more recently on mon-wed-fri. Many but not all options are posted there for each security.

AAPL price data from Kaggle(?)

Treasury Bond rates from home.treasury.gov. Daily treasury par yield rates

In [68]:
import pandas as pd
import numpy as np
#10m rows takes about 30 seconds... Expect long processing times for full data. do in batches.
#f10m = pd.read_csv('pnp_options.csv', nrows=10000000)

f1m = pd.read_csv('pnp_options.csv', usecols=['date', 'act_symbol', 'expiration', 'strike', 'call_put', 'bid', 'ask',
       'vol'] ,nrows = 1000000)

prices = pd.read_csv('HistoricalQuotes.csv')

rates = pd.read_csv('treasury_2019.csv', usecols=['Date', '1 Mo', '2 Mo', '3 Mo', '6 Mo', '1 Yr'])


### NOTE...

**The following data engineering assumes**:

1. we are only interested in AAPL options. if we want to expand, we can simply modify the first line to include more act_symbol values

2. our prices dataframe includes prices covering the entire range of dates for option prices, plus an extra n_prices before the earliest option price, such that we can utilize the n_prices preceding the option pricing in our LSTM model down the line 

3. our rates dataframe also contains treasury rates with dates covering all issuance dates of options.
    
4. all of our option time-to-expiries are closest to 1-3 months, not 6+ months. in the 1m row data the longest time-to-expiry is 62 days. if we see in the full data a time-to-expiry longer than 135 days, we need to add an option to use the 6 month treasury rate in our determine_r function.


**Also**, you will probably want to split this cell into multiple smaller ones when we're working with the full data. some of the actions might be computationally expensive.

In [76]:
df = f1m[f1m['act_symbol'] == 'AAPL']
aapl_prices = prices.copy()
rate = rates.copy()

#remove expiration date, replace with int # of days until expiration
df['days_expiry'] = (pd.to_datetime(df['expiration']) - pd.to_datetime(df['date'])).dt.days
df = df.drop(['expiration'], axis=1)

#format dates on df, aapl_prices, rates to match each other

df['date'] = pd.to_numeric(df['date'].str.replace('-',''))
aapl_prices['Date'] = pd.to_datetime(aapl_prices['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
aapl_prices['Date'] = pd.to_numeric(aapl_prices['Date'].str.replace('-',''))
rate['Date'] = pd.to_datetime(rate['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
rate['Date'] = pd.to_numeric(rate['Date'].str.replace('-',''))

#merge rate with df on date, using forward/backwards fill approach, as some dates of options and treasury prices do not line up...
rate_reindexed = rate.set_index('Date').reindex(df['date'])
rate_reindexed = rate_reindexed.ffill().bfill()
df = pd.merge(df, rate_reindexed, left_on='date', right_index=True, how='left')

#choose risk free rate 'r', based on which treasury rate matures closest to the expiration date of the option.
#Then drop other treasury columns leaving just 'r'
def determine_r(row):
    if row['days_expiry'] < 45:
        return row['1 Mo']
    elif 45 <= row['days_expiry'] < 75:
        return row['2 Mo']
    else:
        return row['3 Mo']

df['r'] = df.apply(determine_r, axis=1)
df = df.drop(['1 Mo', '2 Mo', '3 Mo', '6 Mo', '1 Yr'], axis=1)

#Now to the price dataframe...

#create df with n_prices + 1 columns. first column indicating the date on the last pricing. other n_prices columns will be the n_prices leading up to the current date.
#then we will merge again on date, using most recent closing price preceding option pricing.
n = 10
#obtain df of all prices needed for last_n_prices df... and convert closing price to decimal type
minDateIndex = aapl_prices.index[aapl_prices['Date'] == min(df['date'])-1].tolist()[0]
maxDateIndex = aapl_prices.index[aapl_prices['Date'] == max(df['date'])-1].tolist()[0]
prices_all = aapl_prices.loc[maxDateIndex:(minDateIndex+n),].sort_values(by=['Date'])
prices_all[' Close/Last'] = prices_all[' Close/Last'].str.replace('$', '').astype(float)
prices_all = prices_all.drop([' Volume', ' Open', ' High', ' Low'], axis=1)

stepData = []
for i in range(len(prices_all) - n):
    date = prices_all['Date'].iloc[i + n] 
    n_prices = prices_all[' Close/Last'].iloc[i:i + n + 1].tolist() 
    stepData.append([date] + n_prices)
columns = ['Date'] + [f't{i+1}' for i in range(n)] + ['currentP']
lastn_prices = pd.DataFrame(stepData, columns=columns)

df = pd.merge_asof(df, lastn_prices, left_on='date', right_on='Date', direction='backward')

#Create column for 'fair price' of option, just average of bid and ask... then drop out the rows not being used in this first iteration...
df['option_fp'] = (df['ask'] + df['bid'])/2
inputs_df = df.drop(['date', 'act_symbol', 'bid', 'ask', 'vol', 'Date'], axis=1)


/var/folders/49/x210g1j14bs717pp0q4xf3zr0000gq/T/ipykernel_24743/2456644849.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_expiry'] = (pd.to_datetime(df['expiration']) - pd.to_datetime(df['date'])).dt.days


**Preparing Train/Test Data**

In [114]:
from sklearn.model_selection import train_test_split

#y is option_fp
#train and test calls and puts seperately (obviously)
display(inputs_df.head())

call_df = inputs_df[inputs_df['call_put'] == "Call"]
call_df = call_df.drop(['call_put'], axis=1)
put_df = inputs_df[inputs_df['call_put'] == "Put"]
put_df = put_df.drop(['call_put'], axis=1)

CALL_X_train, CALL_X_test, CALL_Y_train, CALL_Y_test = train_test_split(call_df.drop(['option_fp'], axis=1).values, call_df['option_fp'].values, 
                                                                        test_size=.1, random_state=1)

PUT_X_train, PUT_X_test, PUT_Y_train, PUT_Y_test = train_test_split(put_df.drop(['option_fp'], axis=1).values, put_df['option_fp'].values, 
                                                                        test_size=.1, random_state=1)

#print(CALL_X_train)
#print(CALL_Y_train)
#for input to LSTM-MLP, must split inputs into state inputs (for LSTM), and non-state inputs. We can pass the state inputs seperately through the
#LSTM, then take the LSTM output and concatenate it with the remaining non-state inputs. Thus we must split the inputs into two list elements,
#the first of which being a list of state-inputs (all price data), the second of which being a list of the remaining inputs.

#Note... Should we put the current asset price as part of the state inputs, or the non-state inputs?? try both....
#let's start with putting all pricing info in the state:
CALL_X_train = [CALL_X_train[:,3:].reshape(CALL_X_train.shape[0],n+1,1), CALL_X_train[:,:3]]
CALL_X_test = [CALL_X_test[:,3:].reshape(CALL_X_test.shape[0],n+1,1), CALL_X_test[:,:3]]
PUT_X_train = [PUT_X_train[:,3:].reshape(PUT_X_train.shape[0],n+1,1), PUT_X_train[:,:3]]
PUT_X_test = [PUT_X_test[:,3:].reshape(PUT_X_test.shape[0],n+1,1), PUT_X_test[:,:3]]

,strike,call_put,days_expiry,r,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,currentP,option_fp
0,145.0,Call,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
1,145.0,Call,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
2,145.0,Call,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
3,145.0,Call,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
4,145.0,Call,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675


**Model Params**

In [130]:
layers = 4
features = 3
n_batch = 4096
n_epochs = 100

**Building LSTM-MLP Model**

In [131]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils import plot_model

def make_model():
    close_history = Input((n+1, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=8, input_shape=(n+1, 1), return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

In [132]:
call_model = make_model()
call_model.summary()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 11, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 8)         │      1,952 │ input_layer_7[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 11)        │          0 │ sequential_2[0][… │
│ (Concatenate)       │                   │            │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 100)       │      1,200 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 100)       │        400 │ dense_8[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_6       │ (None, 100)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 100)       │     10,100 │ leaky_re_lu_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 100)       │        400 │ dense_9[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_7       │ (None, 100)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 100)       │     10,100 │ leaky_re_lu_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 100)       │        400 │ dense_10[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_8       │ (None, 100)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │        101 │ leaky_re_lu_8[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,653 (96.30 KB)

 Trainable params: 24,053 (93.96 KB)

 Non-trainable params: 600 (2.34 KB)

In [135]:
call_model.compile(optimizer=Adam(learning_rate=1e-2), loss='mse')
history = call_model.fit(CALL_X_train, CALL_Y_train, 
                    batch_size=n_batch, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
call_model.save('saved-models/call_test1.keras')

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 166ms/step - loss: 44.1166 - val_loss: 140.7040
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 41.8735 - val_loss: 127.3271
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 41.3952 - val_loss: 96.1940
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 40.9006 - val_loss: 50.6137
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - loss: 41.6699 - val_loss: 55.9326
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 41.5289 - val_loss: 59.9196
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - loss: 41.2971 - val_loss: 52.7966
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 41.2307 - val_loss: 50.6133
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 41.1577 - val_loss: 48.6825
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 41.1455 - val_loss: 49.6182


In [136]:
call_model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
history = call_model.fit(CALL_X_train, CALL_Y_train, 
                    batch_size=n_batch, epochs=5, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
call_model.save('saved-models/call_test2.keras')

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 142ms/step - loss: 41.3735 - val_loss: 49.4480
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - loss: 41.3144 - val_loss: 47.9436
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - loss: 40.8607 - val_loss: 35.1794
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 22.0012 - val_loss: 204.1225
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 16.4962 - val_loss: 76.1326


In [137]:
call_model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')
history = call_model.fit(CALL_X_train, CALL_Y_train, 
                    batch_size=n_batch, epochs=5, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
call_model.save('saved-models/call_test3.keras')

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 192ms/step - loss: 14.9334 - val_loss: 82.6164
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 14.7362 - val_loss: 71.6061
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 14.8981 - val_loss: 65.7977
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - loss: 14.6859 - val_loss: 54.9057
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - loss: 14.3615 - val_loss: 49.1139
